# 프로퍼티, 속성과 객체 메서드의 다른 타입들
python은 public, private, protected 프로퍼티를 가지는 다른 언어들과는 다르게 public만 존재함
강제사항은 없으나 밑줄로 시작하는 속성은 해당 객체에 대해 private을 의미 (외부에서 호출하지 않기를 기대하는 것이며 금지하는 것은 아님)

In [2]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60
        
conn = Connector("postgresql://localhost")
print(conn.source)
print(conn._timeout)
print(conn.__dict__)

postgresql://localhost
60
{'source': 'postgresql://localhost', '_timeout': 60}


위의 코드에서 source는 public이고 timeout은 private이나 실제로는 두가지 속성에 모두 접근 가능

#### 일부 속성을 private로 만들 수 있다는 오해: 이중 밑줄 이용

In [3]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60
        
    def connect(self):
        print("connecting with {0}s".format(self.__timeout))
        
conn = Connector("postgresql://localhost")
conn.connect()
print(conn.__timeout)

connecting with 60s


AttributeError: 'Connector' object has no attribute '__timeout'

위처럼 timeout 속성을 숨겼으므로 timeout이 private이라고 생각할 수 있으나 위에서 난 에러는 Attribute에러로 __timeout이라는 속성이 존재하지 않는다는 내용  
밑줄 두개를 이용하면 파이썬은 다른 이름을 만듦: **이름 맹글링**
- 만들어지는 속성의 이름 = "_class-name__attribute-name" = 위의 경우 "&#95;Connector&#95;&#95;timeout"

In [4]:
print(vars(conn))
print(conn._Connector__timeout)
conn._Connector__timeout = 30
print(conn._Connector__timeout)

{'source': 'postgresql://localhost', '_Connector__timeout': 60}
60
30


이름 맹글링을 통해 만들어진 이름으로 해당 변수에 접근 가능  

**이중 밑줄을 사용하는 이유**  
&emsp;= 여러번 확장되는 클래스의 매서드를 이름 충돌 없이 오버라이드 하기 위함  
    
이중 밑줄은 파이썬스러운 코드가 아니므로 속성을 private으로 정의하려는 경우 하나의 밑줄을 사용할 것!

### 프로퍼티
객체에 값을 저장하거나 객체의 상태나 다른 속성의 값을 기반으로 계산을 수행하려고 할 때 프로퍼티를 사용하는 것은 좋은 선택  

**프로퍼티**: 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용  
- 다른 프로그래밍 언어와 달리 접근 메서드(getter, setter)를 사용하지 않고 프로퍼티를 사용
- 프로퍼티는 명렁-쿼리 분리 원칙(command and query separation, 상태 변경과 값 반환을 위한 객체의 메서드는 따로 있어야 한다)을 따르기 좋은 방법
- @property 데코레이터 = 무언가에 응답하기 위한 쿼리
- @proptert_name.setter = 무언가를 하기 위한 커맨드

In [12]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None
        
    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError("유효한 이메일이 아니므로 {0} 값을 사용할 수 없음".format(new_email))
        self._email = new_email

In [13]:
u1 = User("jsmith")
u1.email = "jsmith@"

ValueError: 유효한 이메일이 아니므로 jsmith@ 값을 사용할 수 없음

In [14]:
u1.email = "jsmith@g.co"
print(u1.email)

jsmith@g.co
